In [ ]:
import pandas as pd

In [ ]:
%%capture
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6"


Downloading...
From: https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6
To: /content/Eval.zip
100% 1.29M/1.29M [00:00<00:00, 126MB/s]


In [ ]:
! unzip /content/Eval.zip

Archive:  /content/Eval.zip
   creating: Eval/
   creating: Eval/model_base_7/
  inflating: Eval/model_base_7/base_model_outputs.csv  
  inflating: Eval/model_base_7/metrics.json  
  inflating: Eval/model_base_7/desktop.ini  
   creating: Eval/model_base_1/
  inflating: Eval/model_base_1/metric base model 1.json  
  inflating: Eval/model_base_1/base_model_outputs.csv  
  inflating: Eval/model_base_1/desktop.ini  
   creating: Eval/model_ft_1/
  inflating: Eval/model_ft_1/fine tune base model.json  
  inflating: Eval/model_ft_1/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_1/desktop.ini  
   creating: Eval/model_ft_7/
  inflating: Eval/model_ft_7/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_7/metrics.json  
  inflating: Eval/model_ft_7/desktop.ini  
   creating: Eval/model_base_8/
  inflating: Eval/model_base_8/base_model_outputs.csv  
  inflating: Eval/model_base_8/metrics.json  
  inflating: Eval/model_base_8/desktop.ini  
   creating: Eval/model_base_5/
   creating: Eval

In [ ]:
# prompt: import bert score, bleu, rougr
%%capture
!pip install bert-score
!pip install evaluate
!pip install rouge_score

In [ ]:
import bert_score
from evaluate import load
import pandas as pd

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

predictions = ["the cat is on the mat"]
references = [["there is a cat on the mat"]]

# BLEU-1
result_bleu1 = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {result_bleu1['bleu']:.4f}")

# BLEU-2
result_bleu2 = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {result_bleu2['bleu']:.4f}")

# BLEU-3
result_bleu3 = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {result_bleu3['bleu']:.4f}")

# BLEU-4 (default)
result_bleu4 = bleu.compute(predictions=predictions, references=references)
print(f"BLEU-4: {result_bleu4['bleu']:.4f}")


BLEU-1: 0.7054
BLEU-2: 0.4887
BLEU-3: 0.3697
BLEU-4: 0.0000


In [ ]:
pd.options.display.max_colwidth = 500

Function Compute Bleu score


In [ ]:
import evaluate
import pandas as pd

# Load evaluator sekali saja
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_bleu_per_row(predictions, references):
    """
    Menghitung BLEU 1-4 untuk setiap pasangan prediksi dan referensi.
    Return: list of dict, tiap dict {'bleu1':..., 'bleu2':..., 'bleu3':..., 'bleu4':...}
    """
    all_scores = []
    for pred, ref in zip(predictions, references):
        scores = {}
        # ref harus list of references, meskipun 1 referensi
        ref_list = [ref] if isinstance(ref, str) else ref
        for n in range(1, 5):
            score = bleu.compute(predictions=[pred], references=[ref_list], max_order=n, smooth=True)
            scores[f'bleu{n}'] = score['bleu']
        all_scores.append(scores)
    return all_scores

Function Compoute Rouge Score

In [ ]:
def compute_rouge_per_row(predictions, references):
    rouge = evaluate.load("rouge")
    scores = []
    for pred, ref in zip(predictions, references):
        score = rouge.compute(predictions=[pred], references=[ref])
        # Ambil langsung fmeasure-nya (hasil evaluate adalah dict float)
        scores.append({
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLSum": score["rougeLsum"]
        })
    return scores


Bert score multiligual bert

In [ ]:
import evaluate

def compute_bertscore_per_row(predictions, references):
    """
    Menghitung BERTScore per kalimat untuk Bahasa Indonesia menggunakan library evaluate.

    Parameters:
    - predictions: List[str], list kalimat yang dihasilkan model.
    - references: List[List[str]], list referensi yang diharapkan (nested list, satu referensi per kalimat).

    Returns:
    - List of Dicts berisi precision, recall, dan F1 untuk setiap kalimat.
    """
    # Muat evaluator
    bertscore = evaluate.load("bertscore")

    # Pastikan referensi adalah list of strings (flatten)
    flat_references = [ref[0] for ref in references]

    # Hitung skor
    results = bertscore.compute(
        predictions=predictions,
        references=flat_references,
        lang="id"  # Bahasa Indonesia
    )

    # Gabungkan hasil per kalimat
    scores_per_row = []
    for i in range(len(predictions)):
        scores_per_row.append({
            "prediction": predictions[i],
            "reference": flat_references[i],
            "bert_precision": results["precision"][i],
            "bert_recall": results["recall"][i],
            "bert_f1": results["f1"][i],
        })

    return scores_per_row

# Contoh penggunaan
predictions = ["kucing itu duduk di atas tikar"]
references = [["seekor kucing duduk di tikar"]]

scores = compute_bertscore_per_row(predictions, references)

print("BERTScore per row:")
for i, score in enumerate(scores):
    print(f"Row {i+1}:")
    print(f"  Prediction: {score['prediction']}")
    print(f"  Reference : {score['reference']}")
    print(f"  Precision : {score['bert_precision']:.4f}")
    print(f"  Recall    : {score['bert_recall']:.4f}")
    print(f"  F1        : {score['bert_f1']:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_REMOVED_BY_SCRIPT' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_REMOVED_BY_SCRIPT]` or `pip install hf_REMOVED_BY_SCRIPT`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore per row:
Row 1:
  Prediction: kucing itu duduk di atas tikar
  Reference : seekor kucing duduk di tikar
  Precision : 0.7809
  Recall    : 0.7824
  F1        : 0.7817


# Model 8

## Base Model 8

In [ ]:
base_8 = pd.read_csv('/content/Eval/model_base_8/base_model_outputs.csv')
base_8.head()

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nSolusi atasi sakit maag yang tak kunjung sembuh\n\n### Detail Pertanyaan:\nSolusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali ...",0.679730,0.688819,0.684245
1,solusi untuk mengatasi konstipasi,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nsolusi untuk mengatasi konstipasi\n\n### Detail Pertanyaan:\nsolusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?\n\n### Jawaban:\nSolusi untuk men...",0.650325,0.649686,0.650005
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. saya ingin tany dok.. istri saya mrasakan sakit perut, kpala pusing di sertai mual2 dan bersendawa yang brlebih.. kira-kira apa pnyebab nya dan bgm solusi mngatasiny. terima kasih","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...","Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nPerut sakit, mual dan bersendawa berlebihan\n\n### Detail Pertanyaan:\nPerut sakit, mual dan ber

In [ ]:
import re

def ambil_jawaban_atau_asli(text):
    match = re.search(r'### Jawaban:\s*(.*)', text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return text  # teks asli kalau tidak ada ### Jawaban

In [ ]:
base_8['generated'] = base_8['generated'].apply(ambil_jawaban_atau_asli)

In [ ]:
base_8.head()

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Keluhan Anda seperti sakit maag akut. Anda perlu evaluasi lebih lanjut.\nTes darah perlu dilakukan untuk mengukur kadar asam lambung, pH, dan lain-lain. Selain itu, perlu juga pemeriksaan lain seperti USG abdomen dan pemeriksaan fisik.\nJika memang didiagnosa sakit maag akut, maka Anda perlu minum obat maag. Selain itu, Anda perlu menjaga pola makan dan pola hidup Anda.\nMaag merupakan penyakit yang sering dialami oleh sebagian besar orang Indonesia. Maag merupakan keluhan yang sangat umum. ...",0.679730,0.688819,0.684245
1,solusi untuk mengatasi konstipasi,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"Solusi untuk mengatasi konstipasi bisa dengan cara minum air putih lebih banyak, memperbanyak sayur dan buah, berolahraga secara teratur, dan hindari konsumsi makanan yang dapat memicu konstipasi seperti makanan berminyak, makanan yang mengandung serat tinggi namun kurang air seperti ketela dan kacang-kacangan.",0.650325,0.649686,0.650005
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. saya ingin tany dok.. istri saya mrasakan sakit perut, kpala pusing di sertai mual2 dan bersendawa yang brlebih.. kira-kira apa pnyebab nya dan bgm solusi mngatasiny. terima kasih","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...","Sakit perut, mual dan bersendawa merupakan gejala umum yang dapat timbul akibat gangguan pada organ pencernaan. Namun dapat pula disebabkan oleh gangguan pada organ lain seperti ginjal, hati, paru-paru, dan organ lainnya. Untuk itu perlu dilakukan pemeriksaan fisik secara langsung oleh dokter untuk menentukan penyebab pastinya. Jika penyebabnya adalah gangguan pada organ pencernaan, maka dapat dilakukan pemeriksaan penunjang seperti tes darah, USG atau rontgen perut. Pengobatan yang diberika...",0.645356,0.643648,0.644501
3,Perut sering sak

In [ ]:
base_8.isna().sum()

,0
instruction,0
input,0
reference,0
generated,0
bert_precision,0
bert_recall,0
bert_f1,0


In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base_8.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
df_bert_score

,prediction,reference,bert_precision,bert_recall,bert_f1
0,"Keluhan Anda seperti sakit maag akut. Anda perlu evaluasi lebih lanjut.\nTes darah perlu dilakukan untuk mengukur kadar asam lambung, pH, dan lain-lain. Selain itu, perlu juga pemeriksaan lain seperti USG abdomen dan pemeriksaan fisik.\nJika memang didiagnosa sakit maag akut, maka Anda perlu minum obat maag. Selain itu, Anda perlu menjaga pola makan dan pola hidup Anda.\nMaag merupakan penyakit yang sering dialami oleh sebagian besar orang Indonesia. Maag merupakan keluhan yang sangat umum. ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.694881,0.687795,0.691320
1,"Solusi untuk mengatasi konstipasi bisa dengan cara minum air putih lebih banyak, memperbanyak sayur dan buah, berolahraga secara teratur, dan hindari konsumsi makanan yang dapat memicu konstipasi seperti makanan berminyak, makanan yang mengandung serat tinggi namun kurang air seperti ketela dan kacang-kacangan.",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.675544,0.606564,0.639198
2,"Sakit perut, mual dan bersendawa merupakan gejala umum yang dapat timbul akibat gangguan pada organ pencernaan. Namun dapat pula disebabkan oleh gangguan pada organ lain seperti ginjal, hati, paru-paru, dan organ lainnya. Untuk itu perlu dilakukan pemeriksaan fisik secara langsung oleh dokter untuk menentukan penyebab pastinya. Jika penyebabnya adalah gangguan pada organ pencernaan, maka dapat dilakukan pemeriksaan penunjang seperti tes darah, USG atau rontgen perut. Pengobatan yang diberika...","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...",0.636467,0.628986,0.632705
3,"Dear Sis, sakit perut dan mual bisa disebabkan oleh banyak hal, seperti:\n- gangguan pencernaan\n- gangguan lambung\n- gangguan hormonal\n- gangguan sistem pencernaan\n- gangguan emosi\n- gangguan mental\n- gangguan psikologis\n- gangguan psikiatris\n- gangguan sosial\n- gangguan kejiwaan\n- gangguan kepribadian\n- gangguan kesehatan jiwa\n- gangguan jiwa\n- gangguan mental jiwa\n- gangguan psikis\n- gangguan psikologis\n- gangguan psikiatri\n- gangguan psikososial\n- gangguan psikosomatik\n...","sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala yang anda alami dan melakukan pemeriksaan fisik yang diperlukan. pemeriksaan penunjang seperti tes darah, tes urine, usg mungkin akan dokter anjurkan. terapi yang sesuai akan dokter berikan.anjuran yang dapat anda lakukan: konsumsi makanan bersih dan bergizi secara teratur minum air putih dalam jumlah cukup minimal 10 gelas perhari hindari kebiasaan menahan buang air kecil dan buang air besar manajemen stres s...",0.524433,0.563572,0.543299


In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Keluhan Anda seperti sakit maag akut. Anda perlu evaluasi lebih lanjut.\nTes darah perlu dilakukan untuk mengukur kadar asam lambung, pH, dan lain-lain. Selain itu, perlu juga pemeriksaan lain seperti USG abdomen dan pemeriksaan fisik.\nJika memang didiagnosa sakit maag akut, maka Anda perlu minum obat maag. Selain itu, Anda perlu menjaga pola makan dan pola hidup Anda.\nMaag merupakan penyakit yang sering dialami oleh sebagian besar orang Indonesia. Maag merupakan keluhan yang sangat umum. ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.694881,0.687795,0.691320,30.981595,13.099251,4.731432,2.393006,33.707865,5.639098,12.734082,22.097378
1,"Solusi untuk mengatasi konstipasi bisa dengan cara minum air putih lebih banyak, memperbanyak sayur dan buah, berolahraga secara teratur, dan hindari konsumsi makanan yang dapat memicu konstipasi seperti makanan berminyak, makanan yang mengandung serat tinggi namun kurang air seperti ketela dan kacang-kacangan.",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.675544,0.606564,0.639198,5.242004,2.417956,1.386433,0.887673,23.280423,4.278075,12.698413,12.698413


In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.634338
bert_recall,0.637987
bert_f1,0.635466
bleu1,15.790909
bleu2,6.193110
bleu3,3.071179
bleu4,1.874799
rouge1,20.217796
rouge2,3.355582
rougeL,10.552755


In [ ]:
import os
# os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_8_mean.csv')

In [ ]:
df_scores = pd.concat([base_8[['input']], df_scores], axis=1)
df_scores.head(2)

,input,input,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Keluhan Anda seperti sakit maag akut. Anda perlu evaluasi lebih lanjut.\nTes darah perlu dilakukan untuk mengukur kadar asam lambung, pH, dan lain-lain. Selain itu, perlu juga pemeriksaan lain seperti USG abdomen dan pemeriksaan fisik.\nJika memang didiagnosa sakit maag akut, maka Anda perlu minum obat maag. Selain itu, Anda perlu menjaga pola makan dan pola hidup Anda.\nMaag merupakan penyakit yang sering dialami oleh sebagian besar orang Indonesia. Maag merupakan keluhan yang sangat umum. ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.694881,0.687795,0.691320,30.981595,13.099251,4.731432,2.393006,33.707865,5.639098,12.734082,22.097378
1,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?","solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?","Solusi untuk mengatasi konstipasi bisa dengan cara minum air putih lebih banyak, memperbanyak sayur dan buah, berolahraga secara teratur, dan hindari konsumsi makanan yang dapat memicu konstipasi seperti makanan berminyak, makanan yang mengandung serat tinggi namun kurang air seperti ketela dan kacang-kacangan.",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.675544,0.606564,0.639198,5.242004,2.417956,1.386433,0.887673,23.280423,4.278075,12.698413,12.698413


In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_8.csv')

In [ ]:
df_scores.isna().sum()

,0
input,0
input,0
prediction,0
reference,0
bert_precision,0
bert_recall,0
bert_f1,0
bleu1,0
bleu2,0
bleu3,0


## Fine Tune Model 8

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model_8 = pd.read_csv('/content/Eval/model_ft_8/eval_basemodel_20250525_104038/fine_tuned_outputs.csv')
ft_model_8.head()

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nSolusi atasi sakit maag yang tak kunjung sembuh\n\n### Detail Pertanyaan:\nSolusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali ...",0.685763,0.704031,0.694777
1,solusi untuk mengatasi konstipasi,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nsolusi untuk mengatasi konstipasi\n\n### Detail Pertanyaan:\nsolusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?\n\n### Jawaban:\nkondisi ini dapa...",0.664259,0.711250,0.686952
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. saya ingin tany dok.. istri saya mrasakan sakit perut, kpala pusing di sertai mual2 dan bersendawa yang brlebih.. kira-kira apa pnyebab nya dan bgm solusi mngatasiny. terima kasih","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...","Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nPerut sakit, mual dan bersendawa berlebihan\n\n### Detail Pertanyaan:\nPerut sakit, mual dan ber

In [ ]:
import re

def ambil_jawaban_atau_asli(text):
    match = re.search(r'### Jawaban:\s*(.*)', text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return text  # teks asli kalau tidak ada ### Jawaban

ft_model_8['generated'] = ft_model_8['generated'].apply(ambil_jawaban_atau_asli)

In [ ]:
ft_model_8

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","sebelumnya, perlu diperjelas, sudah berapa lama persisnya anda mengalami penurunan berat badan yang cukup signifikan tersebut? apakah anda juga mengalami muntah berdarah? muntah yang berwarna kekuningan bisa merupakan salah satu gejala gangguan pencernaan, misalnya gastritis , tukak lambung , gerd (gastroesophageal reflux disease), dan sebagainya. muntah berdarah yang muncul disertai dengan penurunan berat badan , jika terjadi sangat , perlu diwaspadai kemungkinannya, yakni tukak lambung, pe...",0.685763,0.704031,0.694777
1,solusi untuk mengatasi konstipasi,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"kondisi ini dapat disebabkan oleh berbagai hal, salah satunya adalah konsumsi makanan yang kurang serat. selain itu, konstipasi dapat pula disebabkan oleh: kurangnya asupan cairan perubahan pola makan atau berat badan yang kurang aktif bergerak atau berolahraga sering menahan bab kondisi medis tertentu, seperti: diabetes hipotiroid hipertiroid stroke penyakit parkinson cedera saraf tulang belakang tumor usus efek samping obat-obatan tertentu, misalnya obat suplemen kalsium, obat antiepilepsi...",0.664259,0.711250,0.686952
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. saya ingin tany dok.. istri saya mrasakan sakit perut, kpala pusing di sertai mual2 dan bersendawa yang brlebih.. kira-kira apa pnyebab nya dan bgm solusi mngatasiny. terima kasih","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...","perut yang terasa nyeri, pusing, mual, dan bersendawa berlebihan mungkin menandakan beberapa kondisi berikut: dispepsia (peningkatan asam lambung) gastritis (radang mukosa lambung) tukak lambung tukak duodenum gastroenteritis (infeksi usus) alergi makanan keracunan makanan infeksi virus atau bakteri, misalnya demam tifoid, hepatitis, meningitis tumor intraa

In [ ]:
ft_model_8.isnull().sum()

,0
instruction,0
input,0
reference,0
generated,0
bert_precision,0
bert_recall,0
bert_f1,0


In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model_8)

In [ ]:
df_ft.head(2)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
0,34.183315,14.456616,7.532923,4.135451,32.644628,4.149378,14.049587,14.049587,73.370367,72.735870,73.051739,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","sebelumnya, perlu diperjelas, sudah berapa lama persisnya anda mengalami penurunan berat badan yang cukup signifikan tersebut? apakah anda juga mengalami muntah berdarah? muntah yang berwarna kekuningan bisa merupakan salah satu gejala gangguan pencernaan, misalnya gastritis , tukak lambung , gerd (gastroesophageal reflux disease), dan sebagainya. muntah berdarah yang muncul disertai dengan penurunan berat badan , jika terjadi sangat , perlu diwaspadai kemungkinannya, yakni tukak lambung, pe..."
1,38.190955,24.055162,16.021117,11.008549,43.613707,16.300940,29.906542,29.906542,72.268534,74.326497,73.283070,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"kondisi ini dapat disebabkan oleh berbagai hal, salah satunya adalah konsumsi makanan yang kurang serat. selain itu, konstipasi dapat pula disebabkan oleh: kurangnya asupan cairan perubahan pola makan atau berat badan yang kurang aktif bergerak atau berolahraga sering menahan bab kondisi medis tertentu, seperti: diabetes hipotiroid hipertiroid stroke penyakit parkinson cedera saraf tulang belakang tumor usus efek samping obat-obatan tertentu, misalnya obat suplemen kalsium, obat antiepilepsi..."


In [ ]:
df_ft.isnull().sum()

,0
bleu1,0
bleu2,0
bleu3,0
bleu4,0
rouge1,0
rouge2,0
rougeL,0
rougeLSum,0
bert_precision,0
bert_recall,0


In [ ]:
mean_ft

,0
bleu1,28.367484
bleu2,13.878875
bleu3,7.906612
bleu4,5.103034
rouge1,32.215402
rouge2,7.605137
rougeL,16.798836
rougeLSum,16.798836
bert_precision,69.752494
bert_recall,69.737121


In [ ]:
df_ft.to_csv('Hasil Eval revisi/eval_ft_model_8.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_8_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-8.zip /content/Hasil\ Eval\ revisi

updating: content/Hasil Eval revisi/ (stored 0%)
updating: content/Hasil Eval revisi/eval_basemodel_8.csv (deflated 74%)
updating: content/Hasil Eval revisi/eval_ft_model_8.csv (deflated 71%)
updating: content/Hasil Eval revisi/.ipynb_checkpoints/ (stored 0%)
updating: content/Hasil Eval revisi/eval_basemodel_8_mean.csv (deflated 39%)
updating: content/Hasil Eval revisi/eval_ft_model_8_mean.csv (deflated 41%)
